In [ ]:
from pynq import Overlay
from pynq import DefaultIP

In [ ]:
from math import log,ceil
class Data():
    # Maps for conversion ease
    _modSchemes = {"QPSK":0,"BPSK":1,"QAM16":2,"QAM64":3}
    _modSchemeBps = {"BPSK":1,"QPSK":2,"QAM16":4,"QAM64":6}
    def __init__(self):
        # Variables User Sets Up
        self.isUL = None
        self.crcLen = None
        self.K = None
        self.E = None
        self.modScheme = None
        # Internal variables
        self._msgLen = None
        self._iIl = None
        self._iBIL = None
        self._nMax = None
        self._N = None
        self._E_mod = None
        self._isConfigured = False
    
    def _setN(self):
        cl2e = ceil(log(self.E,2))
        if self.E <= 9/8 * 2**(cl2e-1) and self.K/self.E < 9/16:
            n1 = cl2e-1
        else:
            n1 = cl2e
        rmin = 1/8
        n2 = ceil(log(self.K/rmin,2))
        nMin = 5
        n = max(min(n1,n2,self._nMax),nMin)
        self._N = 2^n
    
    def _setE_mod(self):
        self._E_mod = (2* self.E) // Data._modSchemeBps[self.modScheme]
    
    def _setInternalVar(self):
        ''' Setup iIl ,iBL and nMax according to uplink or downlink '''
        if self.isUL:
            self._iIl = False
            self._iBIL = True
            self._nMax = 10
        else:
            self._iIl = True
            self._iBIL = False
            self._nMax = 9
        self._setN()
        self._setE_mod()
        
    

    def _setK(self,K):
        if self.isUL:
            if (K >= 18 and K <= 1023) and not (K >= 26 and K <= 30):
                self.K = K
            else:
                self._reset()
                raise ValueError("K must be between 18 and 1023 for UL")
                
        else:
            if (K >= 36 and K <= 184):
                self.K = K
            else:
                self._reset()
                raise ValueError("K must be between 36 and 184 for DL")
                
    
    def _setCrcLen(self,crcLen):
        if self.isUL:
            if crcLen == 6 or crcLen == 11:
                self.crcLen = crcLen
            else:
                self._reset()
                raise ValueError("crcLen must be 6 or 11 for UL")
                
        else:
            if crcLen == 24:
                self.crcLen = crcLen
            else:
                self._reset()
                raise ValueError("crcLen must be 24 for DL")
                
             
    def _setMsgLen(self):
        self._msgLen = self.K - self.crcLen
    
    def _setupMsgSize(self,K,crcLen):
        if self.isUL == None:
            self._reset()
            raise ValueError("isUL must be set before K and crcLen")
            
        else:
            self._setK(K)
            self._setCrcLen(crcLen)
            self._setMsgLen()
    
    def _setupModScheme(self,modScheme):
        if modScheme.upper() in Data._modSchemes.keys():
            self.modScheme = modScheme
            self._bps = Data._modSchemeBps[self.modScheme]
        else:
            self._reset()
            raise ValueError("modScheme must be one of QPSK,BPSK,QAM16,QAM64")
            pass # Throw Error and call _reset   
    
    
    def _setE(self,E):
        temp = log(E,2)
        temp_int = int(temp)
        if E > self.K and temp == temp_int:
            self.E = E
        else: 
            self._reset()
            raise ValueError("E must be greater than K")
            pass # Throw error call _reset
    
    def _setisUL(self, isUL):
        self.isUL = isUL
    
    def setupData(self,isUL, K,crcLen,E,modScheme):
        # set UL
        self._setisUL(isUL)
        # setup K crcLen and msgSize
        self._setupMsgSize(K,crcLen)
        # setup modulation scheme
        self._setupModScheme(modScheme)
        # setup Rate Matched output Length
        self._setE(E)
        #setup internal variables
        self._setInternalVar()
        #data setup complete
        self._isConfigured = True
    def _reset(self):
        # Variables User Sets Up
        self.isUL = None
        self.crcLen = None
        self.K = None
        self.E = None
        self.modScheme = None
        # Internal variables
        self._msgLen = None
        self._bps = None
        self._iIl = None
        self._iBIL = None
        self._nMax = None
        self._N = None
        self._E_mod = None
        self._isConfigured = False
    
    def __str__(self):
        output = "is UL=" + str(self.isUL)+ "\n"
        output += "K=" + str(self.K) + "\n"
        output += "crcLen=" + str(self.crcLen) + "\n"
        output += "E=" + str(self.E) + "\n"
        output += "modscheme=" + str(self.modScheme) + "\n"
        output += "msgLen=" + str(self._msgLen) + "\n"
        output += "bps=" + str(Data._modSchemeBps.get(self.modScheme)) + "\n"
        output += "iIl=" + str(self._iIl) + "\n"
        output += "iBIL=" + str(self._iBIL) + "\n"
        output += "nMax=" + str(self._nMax) + "\n"
        output += "N=" + str(self._N) + "\n"
        output += "E_mod=" + str(self._E_mod) + "\n"
        return output
    
        


        
        





## Driver for Encoder_IP


In [ ]:
class Encoder_IP(DefaultIP):
    bindto = ['xilinx.com:hls:polarEncoderIP:1.1']
    def __init__(self, description):
        super().__init__(description)
        self._data = Data()
    
    def _writeConfig(self):
        self.register_map.crcLen_V = self._data.crcLen
        self.register_map.msgLen_V = self._data._msgLen
        self.register_map.E_V = self._data.E
        self.register_map.nMax_V = self._data._nMax
        self.register_map.iBIL_V = int(self._data._iBIL)
        self.register_map.iIL_V = int(self._data._iIl)
        self.register_map.modScheme = Data._modSchemes[self._data.modScheme]
    
    def config(self,isUL, K,crcLen,E,modScheme):    
        ''' Writes the configuration to the Encoder IP after the checks are performed over all passed arguments'''
        self._data.setupData(isUL,K,crcLen,E,modScheme)
        self._writeConfig()
        
    

    

    


## Driver for Rate_Recover_IP


In [ ]:
class RateRecover_IP(DefaultIP):
    bindto = ['xilinx.com:hls:RateRecover:1.1']
    def __init__(self, description):
        super().__init__(description)
        self._data = Data()
    def _writeConfig(self):
        pass
    def config(self,isUL, K,crcLen,E,modScheme):    
        ''' Writes the configuration to the Encoder IP after the checks are performed over all passed arguments'''
        self._data.setupData(isUL,K,crcLen,E,modScheme)
        self._writeConfig()


## Driver for Decoder_IP


In [ ]:
class Decoder_IP(DefaultIP):
    bindto = ['xilinx.com:hls:PolarDecoder:1.1']
    def __init__(self, description):
        super().__init__(description)

    def _writeConfig(self):
        pass
    def config(self,isUL, K,crcLen,E,modScheme):    
        ''' Writes the configuration to the Encoder IP after the checks are performed over all passed arguments'''
        self._data.setupData(isUL,K,crcLen,E,modScheme)
        self._writeConfig()


## Create Overlay 


In [ ]:
ol = Overlay("PolarCodes.bit")

In [ ]:
ol?

## Create INT array from Characters


In [ ]:
def char_to_int_array(string):
    bits = ''
    for charater in string:
        temp = ord(charater)
        bits += (bin(temp)[2:]).zfill(8)
    bits_array = [int(bit,2) for bit in bits]
    return bits_array


In [ ]:
encoder_ip = ol.Encoder.polarEncoderIP_0

In [ ]:
encoder_ip.config(True,20,11,2048,'QPSK')